In [1]:
import os
import getpass 
import time
import pandas as pd
from sklearn.model_selection import train_test_split

from teradataml import create_context
from teradataml.context.context import get_connection
from teradataml.dataframe.copy_to import copy_to_sql

In [7]:
p = getpass.getpass(prompt="Vantage password")

Vantage password ·········


In [8]:
engine = create_context(host="192.168.116.128", username="ivsm_user", password=p)
conn = get_connection()
cursor = engine.raw_connection().cursor()

In [4]:
iris_df = pd.read_csv("https://aoa-public-datasets.s3-eu-west-1.amazonaws.com/AoaPmmlDemo/iris_full.csv")

In [5]:
_, test = train_test_split(iris_df, test_size=0.5, random_state=123)
predict_df = test.drop("species", 1)

In [6]:
predict_df.head()

,sepal_length,sepal_width,petal_length,petal_width
72,6.3,2.5,4.9,1.5
112,6.8,3.0,5.5,2.1
132,6.4,2.8,5.6,2.2
88,5.6,3.0,4.1,1.3
37,4.9,3.1,1.5,0.1


In [7]:
copy_to_sql(predict_df, table_name = "iris_to_score", index=True, index_label="idx", if_exists="replace")

In [9]:
pd.read_sql("select * from ivsm_user.aoa_ivsm_models", conn)

,model_version,model_id,project_id,deployed_at,model
0,422ee5c0-5f79-4568-8744-d978a5c5c2a2,6610a168-540e-48f4-8947-4b405aaab264,3a3bfc4d-4458-416d-bb50-10ebad597b22,2020-08-13 07:07:15.440,"b'<?xml version=""1.0"" encoding=""UTF-8"" standal..."
1,01b5d6e7-fbdf-4fa7-9e0a-b5b2876dfc4d,09398d14-c34f-4465-bcd1-cb4e34e20cee,b4b37883-8047-4dcb-a11d-acd357f68645,2020-08-11 19:06:38.930,"b'<?xml version=""1.0"" encoding=""UTF-8"" standal..."
2,2eec85eb-8202-4f8d-b092-ad05cc652c37,09398d14-c34f-4465-bcd1-cb4e34e20cee,3a3bfc4d-4458-416d-bb50-10ebad597b22,2020-08-13 05:06:50.520,"b'<?xml version=""1.0"" encoding=""UTF-8"" standal..."


In [9]:
pd.read_sql("select * from aoa_model_09398d14", conn)

,model_id,model
0,09398d14-c34f-4465-bcd1-cb4e34e20cee,"b'<?xml version=""1.0"" encoding=""UTF-8"" standal..."


In [10]:
#clean before fill
cursor.execute("delete from ivsm_user.iris_scoring_results")

ts = time.time()

query="""
insert into ivsm_user.iris_scoring_results
select * from ivsm.IVSM_SCORE(
    on ivsm_user.iris_to_score
    on ivsm_user.aoa_model_09398d14 DIMENSION
    using
        ModelID('09398d14-c34f-4465-bcd1-cb4e34e20cee')
        ColumnsToPreserve('idx')
        ModelType('PMML')
        ModelSpecificSettings('PMML_OUTPUT_TYPE=ALL')
) sc;

"""

cursor.execute(query)

print("Time, spent for scoring (seconds):", "%.2f"%(time.time() - ts))

Time, spent for scoring (seconds): 0.10


In [12]:
# Aggregate output and JSON access


query="""
select
cast(score_result as json).JSONExtractValue('$.target_fields.y') as "species",
count(*) as cnt
from 
ivsm_user.iris_scoring_results
group by 1
"""

pd.read_sql(query, conn)

,species,cnt
0,Iris-virginica,28
1,Iris-setosa,27
2,Iris-versicolor,20
